In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("SQL queries").getOrCreate()

In [7]:
df = spark.read.format("csv").option("header","false").option("inferSchema","true").load("utilization.csv")
df.show(10)

+-------------------+---+----+----+---+
|                _c0|_c1| _c2| _c3|_c4|
+-------------------+---+----+----+---+
|03/05/2019 08:06:14|100|0.57|0.51| 47|
|03/05/2019 08:11:14|100|0.47|0.62| 43|
|03/05/2019 08:16:14|100|0.56|0.57| 62|
|03/05/2019 08:21:14|100|0.57|0.56| 50|
|03/05/2019 08:26:14|100|0.35|0.46| 43|
|03/05/2019 08:31:14|100|0.41|0.58| 48|
|03/05/2019 08:36:14|100|0.57|0.35| 58|
|03/05/2019 08:41:14|100|0.41| 0.4| 58|
|03/05/2019 08:46:14|100|0.53|0.35| 62|
|03/05/2019 08:51:14|100|0.51| 0.6| 45|
+-------------------+---+----+----+---+
only showing top 10 rows



In [9]:
df = df.withColumnRenamed("_c0","event_datetime") \
       .withColumnRenamed("_c1","server_id") \
       .withColumnRenamed("_c2", "free_memory") \
        .withColumnRenamed("_c3","cpu_utilization") \
        .withColumnRenamed("_c4", "session_count")

In [10]:
df.show(10)

+-------------------+---------+-----------+---------------+-------------+
|     event_datetime|server_id|free_memory|cpu_utilization|session_count|
+-------------------+---------+-----------+---------------+-------------+
|03/05/2019 08:06:14|      100|       0.57|           0.51|           47|
|03/05/2019 08:11:14|      100|       0.47|           0.62|           43|
|03/05/2019 08:16:14|      100|       0.56|           0.57|           62|
|03/05/2019 08:21:14|      100|       0.57|           0.56|           50|
|03/05/2019 08:26:14|      100|       0.35|           0.46|           43|
|03/05/2019 08:31:14|      100|       0.41|           0.58|           48|
|03/05/2019 08:36:14|      100|       0.57|           0.35|           58|
|03/05/2019 08:41:14|      100|       0.41|            0.4|           58|
|03/05/2019 08:46:14|      100|       0.53|           0.35|           62|
|03/05/2019 08:51:14|      100|       0.51|            0.6|           45|
+-------------------+---------+-------

In [11]:
#register the dataframe as a table

df.createOrReplaceTempView("utilization")

In [13]:
df_sql = spark.sql("select * from utilization limit 10")
df_sql.show(10)

+-------------------+---------+-----------+---------------+-------------+
|     event_datetime|server_id|free_memory|cpu_utilization|session_count|
+-------------------+---------+-----------+---------------+-------------+
|03/05/2019 08:06:14|      100|       0.57|           0.51|           47|
|03/05/2019 08:11:14|      100|       0.47|           0.62|           43|
|03/05/2019 08:16:14|      100|       0.56|           0.57|           62|
|03/05/2019 08:21:14|      100|       0.57|           0.56|           50|
|03/05/2019 08:26:14|      100|       0.35|           0.46|           43|
|03/05/2019 08:31:14|      100|       0.41|           0.58|           48|
|03/05/2019 08:36:14|      100|       0.57|           0.35|           58|
|03/05/2019 08:41:14|      100|       0.41|            0.4|           58|
|03/05/2019 08:46:14|      100|       0.53|           0.35|           62|
|03/05/2019 08:51:14|      100|       0.51|            0.6|           45|
+-------------------+---------+-------

In [14]:
df_sql_filter = spark.sql('select * from utilization where server_id = 120')
df_sql_filter.show()

+-------------------+---------+-----------+---------------+-------------+
|     event_datetime|server_id|free_memory|cpu_utilization|session_count|
+-------------------+---------+-----------+---------------+-------------+
|03/05/2019 08:06:48|      120|       0.66|           0.31|           54|
|03/05/2019 08:11:48|      120|       0.58|           0.38|           64|
|03/05/2019 08:16:48|      120|       0.55|           0.61|           54|
|03/05/2019 08:21:48|      120|        0.7|           0.35|           80|
|03/05/2019 08:26:48|      120|        0.6|           0.39|           71|
|03/05/2019 08:31:48|      120|       0.53|           0.35|           49|
|03/05/2019 08:36:48|      120|       0.73|           0.42|           73|
|03/05/2019 08:41:48|      120|       0.41|            0.6|           72|
|03/05/2019 08:46:48|      120|       0.62|           0.57|           57|
|03/05/2019 08:51:48|      120|       0.67|           0.44|           78|
|03/05/2019 08:56:48|      120|       

In [15]:
df_sql_filter.count()

10000

In [20]:
df_sql_filter = spark.sql("select server_id, session_count from utilization where session_count > 70 and server_id=120 order by session_count desc")
df_sql_filter.show()

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
+---------+-------------+
only showing top 20 rows



In [21]:
df_agg = spark.sql('select count(*) from utilization where session_count > 70')
df_agg.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [24]:
df_agg = spark.sql('select server_id, count(*) from utilization where session_count > 70 group by server_id')
df_agg.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      148|    8027|
|      137|    8248|
|      133|    8583|
|      108|    8375|
|      101|    9808|
|      115|    5284|
|      126|    6365|
|      103|    8744|
|      128|    3719|
|      122|    4885|
|      111|    3093|
|      140|    6163|
|      132|    2048|
|      146|    7072|
|      142|    7084|
|      139|    7383|
|      120|    2733|
|      117|    3605|
|      112|    7425|
|      127|    5961|
+---------+--------+
only showing top 20 rows



In [25]:
df_agg = spark.sql('select server_id, count(*) from utilization where session_count > 70 group by server_id order by count(*) desc')
df_agg.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
|      123|    7918|
|      118|    7913|
|      112|    7425|
|      139|    7383|
|      104|    7366|
|      121|    7084|
|      142|    7084|
|      146|    7072|
|      126|    6365|
|      144|    6220|
+---------+--------+
only showing top 20 rows



In [27]:
df_agg = spark.sql('select server_id,min(session_count), avg(session_count),max(session_count) from utilization where session_count > 70 group by server_id order by count(*) desc')
df_agg.show()

+---------+------------------+------------------+------------------+
|server_id|min(session_count)|avg(session_count)|max(session_count)|
+---------+------------------+------------------+------------------+
|      101|                71| 87.66557911908646|               105|
|      113|                71| 86.96262476109577|               103|
|      145|                71| 86.97732158211522|               103|
|      103|                71| 85.76372369624886|               101|
|      102|                71| 85.71150710458886|               101|
|      133|                71| 85.46720260981009|               100|
|      108|                71|  85.1219104477612|               100|
|      149|                71|  84.9612693050193|                99|
|      137|                71|  85.0061833171678|                99|
|      148|                71| 84.70350068518749|                99|
|      123|                71| 84.53220510229856|                98|
|      118|                71| 84.

In [29]:
from pyspark.sql import Row

In [30]:
df_dup = sc.parallelize([Row(server_name="101 Server", cpu = 85 , session_count = 80), \
                        Row(server_name="101 Server", cpu = 80 , session_count = 90), \
                        Row(server_name="102 Server", cpu = 85 , session_count = 80), \
                        Row(server_name="102 Server", cpu = 85 , session_count = 80)]).toDF()

In [31]:
df_dup.show()

+---+-----------+-------------+
|cpu|server_name|session_count|
+---+-----------+-------------+
| 85| 101 Server|           80|
| 80| 101 Server|           90|
| 85| 102 Server|           80|
| 85| 102 Server|           80|
+---+-----------+-------------+



In [32]:
df_dup.drop_duplicates().show()

+---+-----------+-------------+
|cpu|server_name|session_count|
+---+-----------+-------------+
| 85| 101 Server|           80|
| 80| 101 Server|           90|
| 85| 102 Server|           80|
+---+-----------+-------------+



In [34]:
df_dup.drop_duplicates(['server_name']).show()

+---+-----------+-------------+
|cpu|server_name|session_count|
+---+-----------+-------------+
| 85| 102 Server|           80|
| 85| 101 Server|           80|
+---+-----------+-------------+



In [35]:
from pyspark.sql import SparkSession
from  pyspark.sql.functions import lit
from pyspark.sql.types import StringType

In [36]:
df= sc.parallelize([Row(server_name="101 Server", cpu = 85 , session_count = 80), \
                        Row(server_name="101 Server", cpu = 80 , session_count = 90), \
                        Row(server_name="102 Server", cpu = 85 , session_count = 40), \
                        Row(server_name="103 Server", cpu = 70 , session_count = 80), \
                        Row(server_name="104 Server", cpu = 60 , session_count = 80)]).toDF()

In [38]:
df_na = df.withColumn('na' , lit(None).cast(StringType()))
df_na.show()

+---+-----------+-------------+----+
|cpu|server_name|session_count|  na|
+---+-----------+-------------+----+
| 85| 101 Server|           80|null|
| 80| 101 Server|           90|null|
| 85| 102 Server|           40|null|
| 70| 103 Server|           80|null|
| 60| 104 Server|           80|null|
+---+-----------+-------------+----+



In [40]:
df_na.fillna('a').show()

+---+-----------+-------------+---+
|cpu|server_name|session_count| na|
+---+-----------+-------------+---+
| 85| 101 Server|           80|  a|
| 80| 101 Server|           90|  a|
| 85| 102 Server|           40|  a|
| 70| 103 Server|           80|  a|
| 60| 104 Server|           80|  a|
+---+-----------+-------------+---+



In [41]:
df2 = df_na.fillna('a').union(df_na)
df2.show()

+---+-----------+-------------+----+
|cpu|server_name|session_count|  na|
+---+-----------+-------------+----+
| 85| 101 Server|           80|   a|
| 80| 101 Server|           90|   a|
| 85| 102 Server|           40|   a|
| 70| 103 Server|           80|   a|
| 60| 104 Server|           80|   a|
| 85| 101 Server|           80|null|
| 80| 101 Server|           90|null|
| 85| 102 Server|           40|null|
| 70| 103 Server|           80|null|
| 60| 104 Server|           80|null|
+---+-----------+-------------+----+



In [42]:
df2.na.drop().show()

+---+-----------+-------------+---+
|cpu|server_name|session_count| na|
+---+-----------+-------------+---+
| 85| 101 Server|           80|  a|
| 80| 101 Server|           90|  a|
| 85| 102 Server|           40|  a|
| 70| 103 Server|           80|  a|
| 60| 104 Server|           80|  a|
+---+-----------+-------------+---+



In [44]:
df2.createOrReplaceTempView('na_table')

In [45]:
spark.sql('select * from na_table').show()

+---+-----------+-------------+----+
|cpu|server_name|session_count|  na|
+---+-----------+-------------+----+
| 85| 101 Server|           80|   a|
| 80| 101 Server|           90|   a|
| 85| 102 Server|           40|   a|
| 70| 103 Server|           80|   a|
| 60| 104 Server|           80|   a|
| 85| 101 Server|           80|null|
| 80| 101 Server|           90|null|
| 85| 102 Server|           40|null|
| 70| 103 Server|           80|null|
| 60| 104 Server|           80|null|
+---+-----------+-------------+----+

